# ML Runs - DOES NOT WORK WITH PYTHON > 3.9, this is a nice tool to have in mind for future!
## What is it?
- One for all solution that has orchestration, monitoring, metrics, logging.
- Instead of Building Mlflow tools and then orchestrating it with Airflow you can use MLRuns
- Teacher does not recommend it for strating the journey with ML Ops but it good to know about it for the future in case we dont have ppl to maitain Airflow stack and MlFlow and just want one stack

In [2]:
import mlrun

ModuleNotFoundError: No module named 'mlrun'

Create a new project

In [3]:
# Initialize the MLRun project object
project = mlrun.get_or_create_project("quick-tutorial", "./", user_project=True)

> 2024-07-05 10:43:43,497 [info] Project loaded successfully: {'project_name': 'quick-tutorial'}


Function code: Run the following cell to generate the data prep file 

In [5]:
%%writefile data-prep.py

import pandas as pd
from sklearn.datasets import load_breast_cancer

import mlrun


@mlrun.handler(outputs=["dataset", "label_column"])
def breast_cancer_generator():
    """
    A function which generates the breast cancer dataset
    """
    breast_cancer = load_breast_cancer()
    breast_cancer_dataset = pd.DataFrame(
        data=breast_cancer.data, columns=breast_cancer.feature_names
    )
    breast_cancer_labels = pd.DataFrame(data=breast_cancer.target, columns=["label"])
    breast_cancer_dataset = pd.concat(
        [breast_cancer_dataset, breast_cancer_labels], axis=1
    )

    return breast_cancer_dataset, "label"

Overwriting data-prep.py


Create a serverless function object from the code above, and register it in the project

In [7]:
data_gen_fn = project.set_function(
    "data-prep.py",
    name="data-prep",
    kind="job",
    image="mlrun/mlrun",
    handler="breast_cancer_generator",
)
project.save()  # save the project with the latest config

Run your data processing function and log artifacts

In [9]:
import sklearn
gen_data_run = project.run_function("data-prep", local=True)

> 2024-07-05 10:43:45,955 [info] Storing function: {'name': 'data-prep-breast-cancer-generator', 'uid': '24e6d1aeec1d4f9484b44a6656cdd4ae', 'db': 'http://localhost:8080'}


> 2024-07-05 10:43:49,118 [info] Run execution finished: {'status': 'completed', 'name': 'data-prep-breast-cancer-generator'}


In [34]:
gen_data_run.logs()

('completed', 0)

Print the output dataset artifact (DataItem object) as dataframe

In [12]:
gen_data_run.artifact("dataset").as_df().head()

mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst texture  worst perimeter  worst area  \
0                 0.07871  ...          17.33           184.60      2019.0   
1                 0.05667  ...          23.41           158.80      1956.0   
2                 0.05999  ...          25.53           152.50      1709.0   
3                 0.09744  ...          26.50            98.87       567.7   
4                 0.05883  ...          16.67           152.20      1575.0   

   worst smoothness  worst compactness  worst concavity  worst concave points  \
0            0.1622             0.6656           0.7119                0.2654   
1            0.1238             0.1866           0.2416                0.1860   
2            0.1444             0.4245           0.4504                0.2430   
3            0.2098             0.8663           0.6869                0.2575   
4            0.1374             0.2050           0.4000                0.1625   

   worst symmetry  worst fractal dimension  label  
0          0.4601                  0.11890      0  
1          0.2750                  0.08902      0  
2          0.3613                  0.08758      0  
3          0.6638                  0.17300      0  
4          0.2364                  0.07678      0  

[5 rows x 31 columns]

Train a model using an MLRun built-in Function Hub: MLRun provides a Function Hub that hosts a set of pre-implemented and validated ML, DL, and data processing functions.

Auto-trainer hub function can train an ML model using a variety of ML frameworks; generate various metrics and charts; and log the model along with its metadata into the MLRun model registry.

In [14]:
import plotly

# Import the function
trainer = mlrun.import_function("hub://auto_trainer")

In [15]:
trainer_run = project.run_function(
    trainer,
    inputs={"dataset": gen_data_run.outputs["dataset"]},
    params={
        "model_class": "sklearn.ensemble.RandomForestClassifier",
        "train_test_split_size": 0.2,
        "label_columns": "label",
        "model_name": "cancer",
    },
    handler="train",
)

> 2024-07-05 10:43:50,453 [info] Storing function: {'name': 'auto-trainer-train', 'uid': 'fe3c86eddc1e4be9a36050b29907e904', 'db': 'http://localhost:8080'}
> 2024-07-05 10:43:51,068 [info] Sample set not given, using the whole training set as the sample set
> 2024-07-05 10:43:52,224 [info] training 'cancer'


> 2024-07-05 10:43:56,406 [info] Run execution finished: {'status': 'completed', 'name': 'auto-trainer-train'}


In [32]:
trainer_run.logs()

> 2024-07-05 10:43:51,068 [info] Sample set not given, using the whole training set as the sample set
> 2024-07-05 10:43:52,224 [info] training 'cancer'



('completed', 153)